In [1]:
import os
import numpy as np
import requests
import re
import copy


# Get Input

In [2]:
sessionId = "os.environ["ADVENT_OF_CODE_SESSION_ID"]"

In [3]:
r = requests.get("https://adventofcode.com/2018/day/13/input", cookies={"session": sessionId})

In [4]:
input = r.content

In [5]:
track_and_cars = [line.decode('utf-8') for line in r.content.splitlines()]

In [6]:
test_input = """/->-\        
|   |  /----\\
| /-+--+-\  |
| | |  | v  |
\-+-/  \-+--/
  \------/   """

In [7]:
test_track_and_cars = [line for line in test_input.splitlines()]

In [8]:
def find_cars(track):
    cars = set()
    for i, line in enumerate(track):
        for j, elem in enumerate(line):
            if elem in ["<", ">", "^", "v"]:
                cars.add((i,j, elem, 0))
    return cars


In [9]:
def parse_track(track_and_cars):
    cars = find_cars(track_and_cars)
    track = copy.copy(track_and_cars)
    
    for car in cars:
        replace_track = "."
        line = track[car[0]]
        if car[2] in [">", "<"]:
            new_line = line[:(car[1])] + "-" + line[(car[1]+1):]
        elif car[2] in ["^", "v"]:
            new_line = line[:(car[1])] + "|" + line[(car[1]+1):]
        track[car[0]] = new_line   
    return track, cars
    
    

In [10]:
test_track, test_cars = parse_track(test_track_and_cars)

In [11]:
test_cars

{(0, 2, '>', 0), (3, 9, 'v', 0)}

In [12]:
def print_track(track, cars=None, domain =None):
    track_plus_cars = copy.copy(track)
    if cars is None:
        cars = []
    for car in cars:
        line = track_plus_cars[car[0]]
        new_line = line[:(car[1])] + car[2] + line[(car[1]+1):]
        track_plus_cars[car[0]] = new_line
    if domain is not None :
        for line in track_plus_cars[domain[0]:domain[1]]:
            print(line[domain[2]:domain[3]])
    else:
        for line in track_plus_cars:
            print(line)

In [14]:
print_track(test_track, test_cars)

/->-\        
|   |  /----\
| /-+--+-\  |
| | |  | v  |
\-+-/  \-+--/
  \------/   


# Part 1

In [ ]:
test_track, test_cars = parse_track(test_track_and_cars)

In [22]:
car_elems = ["^", ">", "v", "<"]

In [23]:
# class MineCar:
#     def __init__(self, i_pos, j_pos, sprite, )

In [24]:
def step_forward(track, cars):
    #print("cars: {}".format(cars))
    new_cars = set()
    sorted_cars =  sorted(cars, key = lambda x: (x[0], x[1]))
    
    for k, car in enumerate(sorted_cars):
        i, j , curr_elem, turn = car
        if curr_elem == ">":
            new_i, new_j = (car[0], car[1]+1)
        elif curr_elem == "v":
            new_i, new_j = (car[0]+1, car[1])
        elif curr_elem == "<":
            new_i, new_j = (car[0], car[1]-1)
        elif curr_elem == "^":
            new_i, new_j = (car[0]-1, car[1])
        else:
            print("curr_elem: {}".format(curr_elem))
        for car in sorted_cars[(k+1):]:
            if (car[0],car[1]) == (new_i, new_j):
                print("CollisionA@({},{})".format(new_j, new_i))
                return track, cars, (new_i, new_j)
        for car in new_cars:
            if (car[0],car[1]) == (new_i, new_j):
                print("CollisionB@({},{})".format(new_j, new_i))
                return track, cars, (new_i, new_j)
        
        new_car = (new_i, new_j, curr_elem, turn)
        new_cars.add(new_car)
    #print("new_cars: {}".format(new_cars))
        
    new_new_cars = set()
    for car in new_cars:
    #for car in sorted(new_cars):
        #print("car: {}".format(car))
        track_elem = track[car[0]][car[1]]
        #print("track_elem: {}".format(track_elem))
        curr_elem = car[2]
        new_car_elem = curr_elem
        turn = car[3]
        if track_elem == "/":
            if curr_elem == "^":
                new_car_elem = ">"
            elif curr_elem ==  "<":
                new_car_elem = "v"
            elif curr_elem == "v":
                new_car_elem = "<"
            elif curr_elem ==  ">":
                new_car_elem = "^"
        elif track_elem == "\\":
            if curr_elem == "^":
                new_car_elem = "<"
            elif curr_elem ==  ">":
                new_car_elem = "v"
            elif curr_elem == "v":
                new_car_elem = ">"
            elif curr_elem ==  "<":
                new_car_elem = "^"
        elif track_elem == "+":
            if turn == 0:
                curr_elem_ind = car_elems.index(curr_elem)
                curr_elem_ind = (curr_elem_ind - 1) % len(car_elems)
                new_car_elem = car_elems[curr_elem_ind]
            elif turn == 2:
                curr_elem_ind = car_elems.index(curr_elem)
                curr_elem_ind = (curr_elem_ind + 1) % len(car_elems)
                new_car_elem = car_elems[curr_elem_ind]
            turn = (turn + 1) % 3
            
        #car =  (car[0], car[1], new_car_elem , turn)
        new_car =  (car[0], car[1], new_car_elem , turn)
        new_new_cars.add(new_car)
    #print("new_new_cars: {}".format(new_new_cars))
    return track, new_new_cars, None
    #return track, cars, None
    

In [25]:
test_cars

{(0, 2, '>', 0), (3, 9, 'v', 0)}

In [26]:
test_track[0][2]

'-'

In [27]:
test_track, test_cars = parse_track(test_track_and_cars)
curr_track, curr_cars = copy.copy(test_track), copy.copy(test_cars)
collision = None
curr_track
print("i: 0\n")
print_track(curr_track, curr_cars)
for i in range(1,1500):
    curr_track, curr_cars, collision = step_forward(curr_track, curr_cars)
    if collision is not None: 
        break
    else:
        print("i: {}\r".format(i), end="")
        #print_track(curr_track, curr_cars)
    

i: 0

/->-\        
|   |  /----\
| /-+--+-\  |
| | |  | v  |
\-+-/  \-+--/
  \------/   
CollisionB@(7,3)


In [29]:
# Part 1

In [28]:
track, cars = parse_track(track_and_cars)
curr_track, curr_cars = copy.copy(track), copy.copy(cars)
collision = None
curr_track
domain=None #[82,102, 98,118]
print("i: 0\n")
#print_track(curr_track, curr_cars,domain=domain)
for i in range(1,1500):
    curr_track, curr_cars, collision = step_forward(curr_track, curr_cars)
    if collision is not None: 
        break
    else:
        print("i: {}\r".format(i), end="")
        #print("i: {}".format(i))
        #print_track(curr_track, curr_cars,  domain=domain)
print("i: {}".format(i))

i: 0

CollisionA@(46,18)
i: 176


In [ ]:
sorted(curr_cars, key=lambda x: (x[0],x[1]))

# Part 2

In [67]:
def step_forward_part_2(track, cars):
    #print("cars: {}".format(cars))
    new_cars = set()
    sorted_cars =  sorted(cars, key = lambda x: (x[0], x[1]))
    removed_cars = set()
    for k, car in enumerate(sorted_cars):
        if car not in removed_cars:
            i, j , curr_elem, turn = car
            if curr_elem == ">":
                new_i, new_j = (car[0], car[1]+1)
            elif curr_elem == "v":
                new_i, new_j = (car[0]+1, car[1])
            elif curr_elem == "<":
                new_i, new_j = (car[0], car[1]-1)
            elif curr_elem == "^":
                new_i, new_j = (car[0]-1, car[1])
            else:
                print("curr_elem: {}".format(curr_elem))
            collision = None
            for car in sorted_cars[(k+1):]:
                if (car[0],car[1]) == (new_i, new_j) and car not in removed_cars:
                    #print("CollisionA@({},{})".format(new_j, new_i))
                    collision = car 
                    
            for car in new_cars:
                if (car[0],car[1]) == (new_i, new_j) and car not in removed_cars:
                    #print("CollisionB@({},{})".format(new_j, new_i))
                    #new_cars.remove(car)
                    collision = car 
        
            if not collision:
                new_car = (new_i, new_j, curr_elem, turn)
                new_cars.add(new_car)
            else:
                removed_cars.add(collision)
                if collision is new_cars:
                    new_cars.remove(collision)
                
                
    for car in removed_cars:
        if car in new_cars:
            new_cars.remove(car)
    #print("new_cars: {}".format(new_cars))
        
    new_new_cars = set()
    for car in new_cars:
    #for car in sorted(new_cars):
        #print("car: {}".format(car))
        track_elem = track[car[0]][car[1]]
        #print("track_elem: {}".format(track_elem))
        curr_elem = car[2]
        new_car_elem = curr_elem
        turn = car[3]
        if track_elem == "/":
            if curr_elem == "^":
                new_car_elem = ">"
            elif curr_elem ==  "<":
                new_car_elem = "v"
            elif curr_elem == "v":
                new_car_elem = "<"
            elif curr_elem ==  ">":
                new_car_elem = "^"
        elif track_elem == "\\":
            if curr_elem == "^":
                new_car_elem = "<"
            elif curr_elem ==  ">":
                new_car_elem = "v"
            elif curr_elem == "v":
                new_car_elem = ">"
            elif curr_elem ==  "<":
                new_car_elem = "^"
        elif track_elem == "+":
            if turn == 0:
                curr_elem_ind = car_elems.index(curr_elem)
                curr_elem_ind = (curr_elem_ind - 1) % len(car_elems)
                new_car_elem = car_elems[curr_elem_ind]
            elif turn == 2:
                curr_elem_ind = car_elems.index(curr_elem)
                curr_elem_ind = (curr_elem_ind + 1) % len(car_elems)
                new_car_elem = car_elems[curr_elem_ind]
            turn = (turn + 1) % 3
            
        #car =  (car[0], car[1], new_car_elem , turn)
        new_car =  (car[0], car[1], new_car_elem , turn)
        new_new_cars.add(new_car)
    #print("new_new_cars: {}".format(new_new_cars))
    return track, new_new_cars
    #return track, cars, None
    

In [ ]:
test_input = """/->-\        
|   |  /----\\
| /-+--+-\  |
| | |  | v  |
\-+-/  \-+--/
  \------/   """

In [81]:
part2_test_input = """/>-<\  
|   |  
| /<+-\\
| | | v
\>+</ |
  |   ^
  \<->/"""
part2_test_track_and_cars = [line for line in part2_test_input.splitlines()]

In [83]:
track, cars = parse_track(part2_test_track_and_cars)
cars

{(0, 1, '>', 0),
 (0, 3, '<', 0),
 (2, 3, '<', 0),
 (3, 6, 'v', 0),
 (4, 1, '>', 0),
 (4, 3, '<', 0),
 (5, 6, '^', 0),
 (6, 3, '<', 0),
 (6, 5, '>', 0)}

In [114]:
track, cars = parse_track(track_and_cars)
curr_track, curr_cars = copy.copy(track), copy.copy(cars)
collision = None
step_forward_part_2(curr_track, curr_cars)
for i in range(1,1500000):
    curr_track, curr_cars = step_forward_part_2(curr_track, curr_cars)
    print("i: {}: cars: {}\r".format(i, len(curr_cars)), end="")
    if len(curr_cars) <= 1: 
        break


In [115]:
curr_cars

{(103, 124, '^', 1)}

In [116]:
track, cars = parse_track(part2_test_track_and_cars)
curr_track, curr_cars = copy.copy(track), copy.copy(cars)
collision = None
curr_track
domain=None #[82,102, 98,118]
print("i: {}, cars: {}".format(0, len(curr_cars)))
print_track(curr_track, curr_cars,domain=domain)
step_forward_part_2(curr_track, curr_cars)
for i in range(1,1500000):
    curr_track, curr_cars = step_forward_part_2(curr_track, curr_cars)
    print("i: {}: cars: {}".format(i, len(curr_cars)))
    print_track(curr_track, curr_cars,  domain=domain)
    if len(curr_cars) <= 1: 
        break


i: 0, cars: 9
/>-<\  
|   |  
| /<+-\
| | | v
\>+</ |
  |   ^
  \<->/
i: 1: cars: 3
/---\  
|   |  
| v-+-\
| | | |
\-+-/ |
  |   |
  ^---^
i: 2: cars: 3
/---\  
|   |  
| /-+-\
| v | |
\-+-/ |
  ^   ^
  \---/
i: 3: cars: 1
/---\  
|   |  
| /-+-\
| | | |
\-+-/ ^
  |   |
  \---/


In [113]:
curr_cars

{(4, 6, '^', 0)}